In [1]:
import pandas as pd
import numpy as np

from datetime import timedelta
from datetime import datetime


In [2]:
year = 2024
fecha = datetime.strptime(f'{year}-01-01', '%Y-%m-%d')
fecha_inicio = fecha - timedelta(days=96)
fecha_fin = datetime.strptime(f'{year+1}-01-01', '%Y-%m-%d')

In [8]:
data = pd.read_csv(f'data/{year}.csv')

In [9]:
data = data[data['Actividad_Realizada'] == 'Exploración']
data['Etapa_Fasica'].value_counts()


Etapa_Fasica
Trasciens Congregans    1
Name: count, dtype: int64

In [10]:

data['Resultado'].value_counts()


Resultado
Negativo    4902
Positivo     660
Name: count, dtype: int64

In [6]:
data.drop(data[data['Resultado']=='En proceso'].index, inplace=True)
data['Resultado'].value_counts()

Resultado
Positivo    1780
Name: count, dtype: int64

In [11]:
data_output = data[['Fecha','Latitud','Longitud','Resultado']]
data_output.loc[:,'Fecha'] = data_output['Fecha'].astype('datetime64[s]')
data_output.loc[:,'Fecha_fin'] = data_output['Fecha']
data_output.loc[:,'Fecha_inicio'] = data_output['Fecha'] - timedelta(days=96)
data_output

C:\Users\seder\AppData\Local\Temp\ipykernel_32944\913995912.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_output.loc[:,'Fecha_fin'] = data_output['Fecha']
C:\Users\seder\AppData\Local\Temp\ipykernel_32944\913995912.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_output.loc[:,'Fecha_inicio'] = data_output['Fecha'] - timedelta(days=96)


,Fecha,Latitud,Longitud,Resultado,Fecha_fin,Fecha_inicio
0,2024-01-02 00:00:00,20.87519,-89.19463,Negativo,2024-01-02 00:00:00,2023-09-28 00:00:00
1,2024-01-02 00:00:00,20.88422,-89.34875,Negativo,2024-01-02 00:00:00,2023-09-28 00:00:00
2,2024-01-03 00:00:00,20.86403,-89.33215,Negativo,2024-01-03 00:00:00,2023-09-29 00:00:00
3,2024-01-02 00:00:00,20.89794,-89.43314,Negativo,2024-01-02 00:00:00,2023-09-28 00:00:00
4,2024-01-02 00:00:00,20.83351,-89.30793,Negativo,2024-01-02 00:00:00,2023-09-28 00:00:00
...,...,...,...,...,...,...
6216,2024-09-26 00:00:00,21.14855,-88.85028,Negativo,2024-09-26 00:00:00,2024-06-22 00:00:00
6217,2024-09-26 00:00:00,21.14259,-88.88364,Negativo,2024-09-26 00:00:00,2024-06-22 00:00:00
6218,2024-09-26 00:00:00,21.12988,-88.87181,Negativo,2024-09-26 00:00:00,2024-06-22 00:00:00
6219,2024-09-26 00:00:00,21.12769,-88.85572,Positivo,2024-09-26 00:00:00,2024-06-22 00:00:00


In [12]:

def get_series(folder,field_name,row,names):
    df = pd.read_csv(folder +field_name+'/' +  str(row['Latitud']) + '_' + str(row['Longitud']) + '.csv')
    
    df['date'] = pd.to_datetime(df['date'],format='%Y-%m-%d')  
    df = df.set_index('date').asfreq('D', method='ffill').reset_index()
    
    mask = (df['date'] > row['Fecha_inicio']) & (df['date'] <= row['Fecha'])
    df = df.loc[mask]
    list_f = df[names].values.tolist()
    len_list = len(list_f)  
    if len_list == 0:
        list_f = [0]*96
    if len_list != 96:
        #print(len(list_f))
        for i in range(len_list,96):
            list_f.append(list_f[len_list-1])
        #print(list_f)
        
        #print(len(list_f))
        #print(row)

    return list_f



def get_series_group(folder,field_name,row,names):
    df = pd.read_csv(folder +field_name+'/' +  str(row['Latitud']) + '_' + str(row['Longitud']) + '.csv')
    df['date'] = pd.to_datetime(df['date'],format='%Y%m%d')  
    #df = df.set_index('date').asfreq('D', method='ffill').reset_index()
    mask = (df['date'] > row['Fecha_inicio']) & (df['date'] <= row['Fecha'])
    df = df.loc[mask]
    list_f = df[names].values.tolist()  
    len_list = len(list_f)  
    if len_list == 0:
        list_f = [0]*96
        len_list = len(list_f)  
    if len_list != 96:
        print('len list')
        print(len(list_f))
        for i in range(len_list,96):
            list_f.append(list_f[len_list-1])

        print('len list f')
        print(len(list_f))
        #print(row)

    return list_f

#get_series(data_output.iloc[0])x['NDVI'].tolist()




In [13]:
def get_series(folder,source,variable,row):
    
    df = pd.read_csv(folder + source +'/' +  str(row['Latitud']) + '_' + str(row['Longitud']) + '.csv')
        
    df['date'] = pd.to_datetime(df['date'],format='%Y-%m-%d')  
    
    df = df.set_index('date').asfreq('D', method='ffill').reset_index()
        
    mask = (df['date'] > row['Fecha_inicio']) & (df['date'] <= row['Fecha'])
    df = df.loc[mask]
    list_f = df[variable].values.tolist()
    len_list = len(list_f)  
    if len_list == 0:
        return [0]*96
    if len_list != 96:
        #print(variable)
        #print(len_list)
        for i in range(len_list,96):
            list_f.append(list_f[len_list-1])

    return list_f

In [14]:
print('NDVI')
data_output['NDVI'] = data_output.apply(
    lambda row: get_series(f'gee-coordinates-{year}/', \
    'NDVI', 'NDVI',row), axis=1)

print('LST_Day_1km')
data_output['LST_Day_1km'] = data_output.apply(
    lambda row: get_series(f'gee-coordinates-{year}/',
    'LST_Day_1km','LST_Day_1km',row), axis=1)

print('SOIL')
features2 =  ['Psurf_f_inst','Qair_f_inst','Rainf_tavg','Tair_f_inst','Wind_f_inst','RootMoist_inst']
for feature in features2:
    data_output[feature] = data_output.apply(
        lambda row: get_series(f'gee-coordinates-{year}/', \
        'SOIL',feature,row), axis=1)

print('DAYMET')
features =  ['prcp','srad','tmax','tmin','vp']
for feature in features:
    data_output[feature] = data_output.apply(
            lambda row: get_series(f'gee-coordinates-{year}/',\
            'DAYMET_V4',feature,row), axis=1)

dataset = data_output[features+features2+['LST_Day_1km','NDVI','Resultado']]
dataset.to_csv(f'data/{year}-dataset.csv', index=False) 




NDVI


C:\Users\seder\AppData\Local\Temp\ipykernel_32944\3457276416.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_output['NDVI'] = data_output.apply(


LST_Day_1km


C:\Users\seder\AppData\Local\Temp\ipykernel_32944\3457276416.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_output['LST_Day_1km'] = data_output.apply(


SOIL


C:\Users\seder\AppData\Local\Temp\ipykernel_32944\3457276416.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_output[feature] = data_output.apply(
C:\Users\seder\AppData\Local\Temp\ipykernel_32944\3457276416.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_output[feature] = data_output.apply(
C:\Users\seder\AppData\Local\Temp\ipykernel_32944\3457276416.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

DAYMET


C:\Users\seder\AppData\Local\Temp\ipykernel_32944\3457276416.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_output[feature] = data_output.apply(
C:\Users\seder\AppData\Local\Temp\ipykernel_32944\3457276416.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_output[feature] = data_output.apply(
C:\Users\seder\AppData\Local\Temp\ipykernel_32944\3457276416.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

In [15]:

dataset = pd.read_csv(f'data/{year}-dataset.csv')

features_all = ['LST_Day_1km','NDVI', \
                'Psurf_f_inst','Qair_f_inst','Rainf_tavg','Tair_f_inst','Wind_f_inst','RootMoist_inst', \
                    'prcp','srad','tmax','tmin','vp']

def convert(row,feature,st,end):
    list_parsed = eval(row[feature])[st:end]
    if np.sum(list_parsed) == 0:
        return 0.0
    else:
        return np.mean(list_parsed)

feature_dataset = pd.DataFrame()
feature_dataset['Resultado'] = dataset['Resultado']
for feature in features_all:
    print(feature)
    feature_dataset[feature + '_1'] = dataset.apply(lambda row: convert(row,feature,0,12),axis=1)
    feature_dataset[feature + '_2'] = dataset.apply(lambda row: convert(row,feature,12,24),axis=1)
    feature_dataset[feature + '_3'] = dataset.apply(lambda row: convert(row,feature,24,36),axis=1)
    feature_dataset[feature + '_4'] = dataset.apply(lambda row: convert(row,feature,36,48),axis=1)
    feature_dataset[feature + '_5'] = dataset.apply(lambda row: convert(row,feature,48,60),axis=1)
    feature_dataset[feature + '_6'] = dataset.apply(lambda row: convert(row,feature,60,72),axis=1)
    feature_dataset[feature + '_7'] = dataset.apply(lambda row: convert(row,feature,72,84),axis=1)
    feature_dataset[feature + '_8'] = dataset.apply(lambda row: convert(row,feature,84,96),axis=1)

feature_dataset.to_csv(f'data/{year}-feature-dataset.csv', index=False) 
feature_dataset.describe()

LST_Day_1km
NDVI
Psurf_f_inst
Qair_f_inst
Rainf_tavg
Tair_f_inst
Wind_f_inst
RootMoist_inst
prcp
srad
tmax
tmin
vp


C:\Users\seder\AppData\Local\Temp\ipykernel_32944\3072436313.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  feature_dataset[feature + '_4'] = dataset.apply(lambda row: convert(row,feature,36,48),axis=1)
C:\Users\seder\AppData\Local\Temp\ipykernel_32944\3072436313.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  feature_dataset[feature + '_5'] = dataset.apply(lambda row: convert(row,feature,48,60),axis=1)
C:\Users\seder\AppData\Local\Temp\ipykernel_32944\3072436313.py:23: PerformanceWarning: DataFrame is highly fragment

,LST_Day_1km_1,LST_Day_1km_2,LST_Day_1km_3,LST_Day_1km_4,LST_Day_1km_5,LST_Day_1km_6,LST_Day_1km_7,LST_Day_1km_8,NDVI_1,NDVI_2,...,tmin_7,tmin_8,vp_1,vp_2,vp_3,vp_4,vp_5,vp_6,vp_7,vp_8
count,5562.000000,5562.000000,5562.000000,5562.000000,5562.000000,5562.000000,5562.000000,5562.000000,5562.000000,5562.000000,...,5562.000000,5562.000000,5562.000000,5562.000000,5562.000000,5562.000000,5562.000000,5562.000000,5562.000000,5562.000000
mean,15425.071152,15442.229714,15442.576651,15432.429357,15406.801555,15370.040603,15327.218282,15295.069325,4981.303788,4921.394043,...,2.284417,2.172369,402.794478,378.719743,353.274926,324.143148,299.562812,281.030802,261.921993,248.280704
std,245.542203,250.783846,262.334645,274.106251,285.971512,290.070657,295.861661,280.097568,1453.534421,1470.700826,...,5.428324,5.130039,952.067433,895.767899,842.720048,780.148717,723.877335,675.863342,623.034070,586.420003
min,14693.500000,14730.750000,14651.083333,14715.000000,14529.000000,14656.916667,14686.166667,14602.000000,313.000000,74.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,15205.937500,15253.208333,15255.291667,15176.000000,15121.166667,15098.020833,15079.333333,15080.270833,3905.791667,3852.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15461.625000,15493.208333,15499.000000,15508.208333,15491.000000,15379.625000,15207.000000,15173.875000,4555.916667,4495.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,15629.083333,15637.645833,15639.375000,15649.625000,15643.166667,15634.083333,15620.812500,15571.770833,5871.500000,5782.375000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,16048.000000,16069.000000,16019.250000,16025.000000,15995.333333,16005.416667,16069.000000,16050.000000,9618.166667,9245.000000,...,19.534167,17.181667,3141.131653,2729.066691,2712.692505,2692.682495,2612.358358,2524.710042,2288.444173,1975.942505


In [21]:
import pandas as pd
df = pd.read_csv('data/2021-feature-dataset.csv') 
df = df.query("Resultado != 'En proceso'")
df.to_csv(f'data/2021-feature-dataset.csv', index=False) 
